### Required libraries :

In [76]:
import pandas as pd  # dataframe structure
import numpy as np  # array structure
import glob, os  # dealing with OS and Files, reading files etc 

### Loading data :

<mark>A noter j'ai supprimé les données concernant 2009 car n'a pas la même forme que les autres ( juste pour lire tout en même temps)</mark>

Dans un premier on va lire tous les fichiers csv qu'on a téléchargé à partir du site datagov :

####  Data set about caracteristiques :

In [79]:
path =r'dataset/caracteristiques' # path containing all csv files about caracteristiques (  use your own path)
allFiles = glob.glob(path + "/*.csv")  # a list containing files names (all csv files)
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1') # read csv file (file_)
    list_.append(df)  # append the the dataframe df to the lust list_
caracteristiques = pd.concat(list_)  # concat all dataframes existing in list_

#####  Data set about usagers :

In [69]:
path =r'dataset/usagers' 
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1')
    list_.append(df)
usagers = pd.concat(list_)

####  Data set about lieux :

In [77]:
path =r'dataset/lieux' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1')
    list_.append(df)
lieux = pd.concat(list_)

C:\Users\pc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Data set about vehicules :

In [71]:
path =r'dataset/vehicules' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='latin-1')
    list_.append(df)
vehicules = pd.concat(list_)

##### Short description about our dataframes :

In [ ]:
print("==== Shape of our dataframes ====")
print("The shape of caracteristiques is :", caracteristiques.shape)
print("The shape of lieux is :", lieux.shape)
print("The shape of vehicules is :", vehicules.shape)
print("The shape of usagers is :", usagers.shape)
print("==== Count Missing Data (only nan , note that 0 means also a nan value but should be converted to nan later durin cleaning stage)  ====")
print("Total missing data in  caracteristiques is :", caracteristiques.isnull().sum().sum())
print("Total missing data in  lieux is :", lieux.isnull().sum().sum())
print("Total missing data in  vehicules is :", vehicules.isnull().sum().sum())
print("Total missing data in  usagers is :", usagers.isnull().sum().sum())

### Data cleaning  :

---
####  La table caracteristiques 
---

Vu que la base de données contient assez d'erreur de saisie, il s'avére que l'étape de nettoyage est primordiale avant de passer à d'autre étapes.

In [24]:
# rappel  sur la taille des données :
caracteristiques.shape
# on 16 variables ( ce qui correspond bien au document pdf) et 765 576 accidents.

(765576, 16)

##### Variables catégorielles :

###### gps :

In [25]:

print(len(caracteristiques.gps.unique()))
caracteristiques.gps.unique()
#On voit que gps a 8 valeurs uniques, alors que dans le documents pdf il n'y a que 5 ( M,A,G,R et Y)

9


array(['M', nan, '0', 'A', 'G', 'R', 'S', 'T', 'Y'], dtype=object)

nan, 0 et S se sont considérées comme des valeurs manquantes

In [26]:
caracteristiques[caracteristiques.gps=='S'].shape

(3, 16)

In [27]:
caracteristiques[caracteristiques.gps=='0'].shape

(9, 16)

In [28]:
caracteristiques.gps.isnull().sum()

432956

In [29]:
# puisque cette variable est utilisée pour localiser l'accident, on va remplacer '0' et'S' par nan.
caracteristiques.loc[:,['gps']]=caracteristiques.loc[:,['gps']].replace('0',np.nan)
caracteristiques.loc[:,['gps']]=caracteristiques.loc[:,['gps']].replace('S',np.nan)

In [30]:
#verification :
print(len(caracteristiques.gps.unique()))
caracteristiques.gps.unique()

7


array(['M', nan, 'A', 'G', 'R', 'T', 'Y'], dtype=object)

In [31]:
caracteristiques.gps=caracteristiques.gps.astype(str) # on change ty type car nan est float alors que la variable est un object

In [32]:
caracteristiques[caracteristiques.gps=='nan'].head() # pour voir le lien entre gps et dep

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
31,200500000032,5,1,18,900,1,2,0,3.0,3.0,770.0,NaN,nan,NaN,NaN,620
42,200500000043,5,1,21,700,3,1,2,1.0,1.0,423.0,NaN,nan,NaN,NaN,620
105,200500000106,5,1,9,230,5,1,2,1.0,6.0,143.0,NaN,nan,NaN,NaN,600
135,200500000136,5,1,12,2145,5,2,2,1.0,7.0,612.0,RUE DE MEAUX,nan,NaN,NaN,600
136,200500000137,5,1,14,745,2,2,3,2.0,3.0,138.0,ROUTE DE COMPIEGNE,nan,NaN,NaN,600


In [33]:
caracteristiques[caracteristiques.gps=='nan'].tail() # pour voir le lien ente gps et dep

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
59427,201600059428,16,12,1,1201,1,2,1,1.0,6.0,611.0,passamanty-Vahib ( rout,nan,NaN,NaN,976
59428,201600059429,16,12,3,2215,1,2,1,1.0,7.0,611.0,mamoudzou centre-Adrian,nan,NaN,NaN,976
59429,201600059430,16,12,5,1710,1,2,1,1.0,3.0,611.0,kaweni-Kaweni sud (Z I),nan,NaN,NaN,976
59430,201600059431,16,12,6,1005,1,2,3,1.0,3.0,611.0,kaweni-Kaweni nord ( Z I,nan,NaN,NaN,976
59431,201600059432,16,12,24,635,1,2,6,1.0,3.0,611.0,kaweni-Kaweni sud (Z I),nan,NaN,NaN,976


On peut remarquer que il y a des nan dans la france metropole et la france d'outre mer.

-Donc pour France Metropole on remplace gps par M ( il suffit de voir dep<950)

-Donc pour la france d'outre mer il suffit de voir aussi dep.

In [34]:
# France M :
caracteristiques.ix[caracteristiques.dep<=950,'gps']=caracteristiques.ix[caracteristiques.dep<=950,'gps'].replace('nan','M')

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [35]:
caracteristiques.ix[caracteristiques.dep==971,'gps']=caracteristiques.ix[caracteristiques.dep==971,'gps'].replace('nan','A')
caracteristiques.ix[caracteristiques.dep==972,'gps']=caracteristiques.ix[caracteristiques.dep==972,'gps'].replace('nan','A')
caracteristiques.ix[caracteristiques.dep==973,'gps']=caracteristiques.ix[caracteristiques.dep==973,'gps'].replace('nan','G')
caracteristiques.ix[caracteristiques.dep==974,'gps']=caracteristiques.ix[caracteristiques.dep==974,'gps'].replace('nan','R')
caracteristiques.ix[caracteristiques.dep==976,'gps']=caracteristiques.ix[caracteristiques.dep==976,'gps'].replace('nan','Y')

Donc cette variable est bien imputée.
Il reste juste le fait de fixer le type de cette variable. Elle est déjà 'object'.

###### jour :

In [36]:
print(len(caracteristiques.jour.unique()))
caracteristiques.jour.unique()
# pas de soucis pour cette variable

31


array([12, 21,  4, 10, 28,  3, 18, 25, 29, 23, 11,  1, 30, 19,  9, 31, 15,
       13, 26,  2,  6,  8, 20, 16,  7,  5, 24, 14, 17, 22, 27], dtype=int64)

###### mois :

In [37]:
print(len(caracteristiques.mois.unique()))
caracteristiques.mois.unique()
# pas de soucis pour cette variable


12


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

##### an :

In [38]:
print(len(caracteristiques.an.unique()))
caracteristiques.an.unique()
#pareil

11


array([ 5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16], dtype=int64)

###### hrmn: 

In [39]:
caracteristiques.hrmn.min(), caracteristiques.hrmn.max()
# donc ici il s'agit de l'heure et les minutes
# exemple : 2359 ==> 23:59
# on cherche le len des éléments 
mes_len=[len(ele) for ele in set(list(caracteristiques.hrmn.astype(str).values))]
[ele for ele in set(mes_len)]
# Il y a donc différents len ( de 1 ==>4) 
# donc if faut du traitement
# si len == 4 ==> ajouter : entre les deux chiffres
# si len == 3 ==> ajouter 0 en avant puis : entre les deux chiffres
# si len == 2 ==> ajouter deux 00 vers la fin puis : 
# si len == 1 ==> ajouter 0 avant et 00 à la fin puis : 

[1, 2, 3, 4]

In [171]:
# change type to str for easy handling :
caracteristiques.hrmn=caracteristiques.hrmn.astype(str)

In [181]:
# len ==4 : split with len=2 then add :
caracteristiques.ix[caracteristiques.hrmn.str.len()==4,'hrmn']= caracteristiques.ix[caracteristiques.hrmn.str.len()==4,'hrmn'].str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [189]:
# len == 3 :  add 0 to into the start then split then add :
caracteristiques.ix[caracteristiques.hrmn.str.len()==3,'hrmn']=('0'+caracteristiques.ix[caracteristiques.hrmn.str.len()==3,'hrmn']).str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [190]:
#len =2 : add 00 to the end then split then add : 
caracteristiques.ix[caracteristiques.hrmn.str.len()==2,'hrmn']=(caracteristiques.ix[caracteristiques.hrmn.str.len()==2,'hrmn'] +'00').str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [191]:
# len == 1 ==> ajouter 0 avant et 00 à la fin puis : 
caracteristiques.ix[caracteristiques.hrmn.str.len()==1,'hrmn']=('0'+caracteristiques.ix[caracteristiques.hrmn.str.len()==1,'hrmn'] +'00').str.extract('(.{2,2})' * 2).apply(lambda x: ':'.join(x), axis=1)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [193]:
# verification :
mes_len_apres=[len(ele) for ele in set(list(caracteristiques.hrmn.astype(str).values))]
[ele for ele in set(mes_len_apres)]
# c'est bon ( len==5 car on a ajouté les :)

[5]

Création d'une nouvelle variables date qui regroupe l'année , le mois , le jour et l'heure'

In [202]:
# il faut encore du traitement 
# add 2000 à la date 
caracteristiques.loc[:,['an']]=caracteristiques.loc[:,['an']]+2000


In [203]:
# add 0 to start of month if len ==1
caracteristiques.mois=caracteristiques.mois.astype(str)
caracteristiques.ix[caracteristiques.mois.str.len()==1,'mois']=('0'+caracteristiques.ix[caracteristiques.mois.str.len()==1,'mois'])

###### Date :

In [220]:
#caracteristiques.an=caracteristiques.an.astype(str)
#caracteristiques.jour=caracteristiques.jour.astype(str)
#caracteristiques.date_accident=caracteristiques.loc[:,['an','mois','jour']].apply(lambda x : '-'.join(x), axis=1)

###### lum :

In [40]:
print(len(caracteristiques.lum.unique()))
caracteristiques.lum.unique()
# bon

5


array([3, 1, 5, 4, 2], dtype=int64)

In [41]:
# change type to object
caracteristiques.lum=caracteristiques.lum.astype(str)

###### agg


In [42]:
print(len(caracteristiques['agg'].unique()))
caracteristiques['agg'].unique()
#bon

2


array([2, 1], dtype=int64)

In [43]:
caracteristiques.loc[:,['agg']]=caracteristiques.loc[:,['agg']].astype(str)

###### intersection :

In [44]:
print(len(caracteristiques.int.unique()))
caracteristiques.int.unique()

10


array([1, 2, 9, 0, 6, 8, 3, 4, 7, 5], dtype=int64)

In [45]:
# 0 est un intrus 
caracteristiques[caracteristiques.int==0].shape
# il y 106 valeurs de 0
# on va remplacer cette valeur par la valeur la plus courante 

(106, 16)

In [46]:
caracteristiques.int.astype(str).describe()
# le plus courant c'est 1 avec une fréquence de 547440
# c'est normale car la majorité des accidents n'ont pas lieu sur les intersections ( vitesse très faible générelement)

count     765576
unique        10
top            1
freq      547440
Name: int, dtype: object

In [47]:
# remplacer 0 par 1: 
caracteristiques.loc[:,['int']]=caracteristiques.loc[:,['int']].replace(0,1)

In [48]:
#verifications :
print(len(caracteristiques.int.unique()))
caracteristiques.int.unique()

9


array([1, 2, 9, 6, 8, 3, 4, 7, 5], dtype=int64)

In [49]:
caracteristiques.int=caracteristiques.int.astype(str)

###### atm :

In [50]:
print(len(caracteristiques.atm.unique()))
caracteristiques.atm.unique()
# il y des nan values, de même on va les remplacer par la valeur la plus courante

10


array([  1.,   2.,   3.,   8.,   9.,   7.,   6.,   4.,   5.,  nan])

In [51]:
# taille des nan values dans atm :
print('taille :', caracteristiques.atm.isnull().sum())
# valeur la plus courante dans atm :
caracteristiques.atm.astype(str).describe()
# 1 est la valeur la plus courante ( normale)

taille : 48


count     765576
unique        10
top          1.0
freq      619775
Name: atm, dtype: object

In [52]:
#remplacer nan par 1 :
caracteristiques.loc[:,['atm']]=caracteristiques.loc[:,['atm']].replace(np.nan,1)

In [53]:
#verification :
print(len(caracteristiques.atm.unique()))
caracteristiques.atm.unique()

9


array([ 1.,  2.,  3.,  8.,  9.,  7.,  6.,  4.,  5.])

In [54]:
caracteristiques.atm=caracteristiques.atm.astype(str)

###### collision :

In [55]:
print(len(caracteristiques.col.unique()))
caracteristiques.col.unique()
# nan values are in 

8


array([  3.,   1.,   5.,   6.,   2.,   4.,   7.,  nan])

In [56]:
# taille des nan values dans col :
print('taille :', caracteristiques.col.isnull().sum())
# valeur la plus courante dans col :
caracteristiques.col.astype(str).describe()
# 6 est la valeur la plus courante ( autre collision)

taille : 11


count     765576
unique         8
top          6.0
freq      252916
Name: col, dtype: object

In [57]:
#remplacer nan par 6 :
caracteristiques.loc[:,['col']]=caracteristiques.loc[:,['col']].replace(np.nan,6)

In [58]:
#verification :
print(len(caracteristiques.col.unique()))
caracteristiques.col.unique()

7


array([ 3.,  1.,  5.,  6.,  2.,  4.,  7.])

In [59]:
caracteristiques.col=caracteristiques.col.astype(str)

###### Adresse postale :

In [60]:
print(len(caracteristiques.adr.unique()))
caracteristiques.adr.unique()
# des adresse, donc pas de problémes ( sauf erreur de saisie. on va lire adresse par adresse bien sûr pour s'assurer )

337657


array(['CD41B', 'rue de Lille', nan, ..., 'tsoundzou-KOUALE ( rue d',
       "passamanty-M'hogoni (rue", 'kaweni-GEOLE (la)'], dtype=object)

##### Variables numériques :

In [61]:
# valeur nan par variable
caracteristiques.isnull().sum()

Num_Acc         0
an              0
mois            0
jour            0
hrmn            0
lum             0
agg             0
int             0
atm             0
col             0
com             2
adr        127664
gps             0
lat        436603
long       436607
dep             0
dtype: int64

###### lon/lat

 lat et lon sont des variables numériques. Ils présentent plus de 50% de valeurs manquantes. On les garde comme ça pour l'instant.

###### com :

Chaque dépertement contient des communes. Dans notre base les déparetements sont chiffrés entre 10 et 976. 

RQ :
    - on peut aggréger nos données par département et étudier chaque département tout seul
    - On peut encore ajouter de la granulité et aggréger par commune.


In [62]:
#nan ==> le mode
caracteristiques.com.astype(str).describe()

count     765576
unique       921
top         55.0
freq       31375
Name: com, dtype: object

In [63]:
caracteristiques.loc[:,['com']]=caracteristiques.loc[:,['com']].replace(np.nan,55)

In [223]:
# change type to object :
caracteristiques.com=caracteristiques.com.astype(str)

###### dep

In [64]:
#pas de valeur manquante pour cette variables
print("il y a ",len(caracteristiques.dep.unique()),"départements ")
print(sorted(list(caracteristiques.dep.unique())))


il y a  101 départements 
[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 201, 202, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 971, 972, 973, 974, 976]


In [65]:
dep_values=list(caracteristiques.dep.astype(str).values)

In [66]:
set([dep for dep in dep_values if len(dep)==2])
# on a bien toutes les valeurs correspondantes au dépertement de numéro entre 1 et 9.

{'10', '20', '30', '40', '50', '60', '70', '80', '90'}

In [67]:
print(set([dep for dep in dep_values if len(dep)==3 and dep[-1]!='0' and dep!='202' and dep!='201'])) 

{'972', '974', '973', '976', '971'}


In [68]:
# 201 et 202 remplacent 2A et 2B ( voir lien www.francegene.com/rech-fr/dep-fr.php) : donc pour ces deux valeurs pas de soucis
# mais les autres ils ne se terminent pas par 0 ?
#  normalement il y a 95 departement  d'après le lien ci-dessous, donc 950 doit être la plus grande valeur.
# A voir : soit se sont des nouveau dépertement ? sinon on va les remplacer par la plus courante valeurs.
# en fait les autres numéros sont celles de la france d'outre mer. plus précisement :
# 971 : num départemental de Guadeloupe
# 972 : Maritinique
# 973 : La Guyane 
# 974 : La Réunion
# 976 : Mayotte
# Donc c'est à nous de choisir : est ce que juste la france europééne ou aussi la france d'outre mer.

In [224]:
#change type to str
caracteristiques.dep=caracteristiques.dep.astype(str)

In [69]:
#save table to csv file :
caracteristiques.to_csv('caracteristiques_2005-2016.csv',index=False,)

---
#### la table lieux : 
---

##### variables catégorielles :

In [70]:
# shape :
lieux.shape

(765576, 18)

###### catégorie de route :

In [71]:
print(len(lieux.catr.unique()))
lieux.catr.unique()
# il y des nan values

8


array([  3.,   2.,   4.,   6.,   9.,   5.,   1.,  nan])

In [72]:
# 9 spécifie "autre" de type de route
print(lieux[lieux.catr==9].shape)  
#taille valeur nan
print(lieux.catr.isnull().sum())

(14992, 18)
1


In [73]:
# remplacer nan par 9.( 9 toujours n'est pas connu donc implicitement c'est nan)
lieux.loc[:,['catr']]=lieux.loc[:,['catr']].replace(np.nan,9)

In [74]:
#verification
lieux.catr.isnull().sum()

0

###### régime circulation :

In [75]:
print(len(lieux.circ.unique()))
lieux.circ.unique()
# il y a 0 et nan qui ne sont pas des valeurs pour cette variables

6


array([  2.,   0.,   3.,   4.,   1.,  nan])

In [76]:
print("taille 0",lieux[lieux.circ==0].shape)
print("taille nan",lieux.circ.isnull().sum())

taille 0 (40732, 18)
taille nan 737


In [77]:
lieux.circ.astype(str).describe()
# top est 2( les accidents ont lieu beaucoup sur les  routes nationals)

count     765576
unique         6
top          2.0
freq      485032
Name: circ, dtype: object

In [78]:
# On va remplacer ces deux variables par la plus courante valeur 
lieux.loc[:,['circ']]=lieux.loc[:,['circ']].replace(np.nan,2)
lieux.loc[:,['circ']]=lieux.loc[:,['circ']].replace(0,2)

In [79]:
# verification :
lieux.circ.unique()

array([ 2.,  3.,  4.,  1.])

###### voie réservée :

In [80]:
print(len(lieux.vosp.unique()))
lieux.vosp.unique()
# 0 et nan doivent être changés

5


array([  0.,   1.,   3.,   2.,  nan])

In [81]:
print("taille 0 :",lieux[lieux.vosp==0].shape)
print("taille nan :",lieux.vosp.isnull().sum())
print(lieux.shape)
# on remarque que quasiment toute la colonne est en '0'.
# on remplace nan par 0 et on laisse cette variable.

taille 0 : (717337, 18)
taille nan : 1382
(765576, 18)


In [82]:
lieux.loc[:,['vosp']]=lieux.loc[:,['vosp']].replace(0,np.nan)


###### profil de la route : 

In [83]:
print(len(lieux.prof.unique()))
lieux.prof.unique()

6


array([  1.,   0.,   2.,   3.,   4.,  nan])

In [84]:
print("taille 0 :",lieux[lieux.prof==0].shape)
print("taille nan :",lieux.prof.isnull().sum())
print(lieux.shape)

taille 0 : (58716, 18)
taille nan : 982
(765576, 18)


In [85]:
# on remplace 0 et nan par le mode :
lieux.prof.astype(str).describe()
# top est 1 (Plat)

count     765576
unique         6
top          1.0
freq      578523
Name: prof, dtype: object

In [86]:
lieux.loc[:,['prof']]=lieux.loc[:,['prof']].replace(0,1)
lieux.loc[:,['prof']]=lieux.loc[:,['prof']].replace(np.nan,1)

###### Tracé en plan :
    
    

In [87]:
print(len(lieux.plan.unique()))
lieux.plan.unique()

6


array([  1.,   3.,   2.,   0.,   4.,  nan])

In [88]:
print("taille 0 :",lieux[lieux.plan==0].shape)
print("taille nan :",lieux.plan.isnull().sum())
print(lieux.shape)

taille 0 : (52708, 18)
taille nan : 1001
(765576, 18)


In [89]:
# on remplace 0 et nan par le mode :
lieux.plan.astype(str).describe()
# top est 1 ( Partie rectiligne)

count     765576
unique         6
top          1.0
freq      583560
Name: plan, dtype: object

In [90]:
lieux.loc[:,['plan']]=lieux.loc[:,['plan']].replace(0,1)
lieux.loc[:,['plan']]=lieux.loc[:,['plan']].replace(np.nan,1)

###### Etat de la surface 

In [91]:
print(len(lieux.surf.unique()))
lieux.surf.unique()

11


array([  1.,   2.,   0.,   9.,   7.,   8.,   5.,   6.,   3.,   4.,  nan])

In [92]:
print("taille 0 :",lieux[lieux.surf==0].shape)
print("taille nan :",lieux.surf.isnull().sum())
print(lieux.shape)

taille 0 : (24885, 18)
taille nan : 949
(765576, 18)


In [93]:
# on remplace par 9 (autre: 
lieux.loc[:,['surf']]=lieux.loc[:,['surf']].replace(0,9)
lieux.loc[:,['surf']]=lieux.loc[:,['surf']].replace(np.nan,9)

###### situation de l'acccident

In [94]:
print(len(lieux.situ.unique()))
lieux.situ.unique()

7


array([  1.,   5.,   3.,   4.,   0.,   2.,  nan])

In [95]:
print("taille 0 :",lieux[lieux.situ==0].shape)
print("taille nan :",lieux.situ.isnull().sum())
print(lieux.shape)

taille 0 : (39296, 18)
taille nan : 943
(765576, 18)


In [96]:
# on remplace par le mode :
lieux.situ.astype(str).describe()
#top est 1 ( sur chaussé)

count     765576
unique         7
top          1.0
freq      658335
Name: situ, dtype: object

In [97]:

lieux.loc[:,['situ']]=lieux.loc[:,['situ']].replace(0,1)
lieux.loc[:,['situ']]=lieux.loc[:,['situ']].replace(np.nan,1)

###### point école :

In [98]:
# on sait pas pour cette variables quelles sont les veleurs prises
lieux.env1.unique()
# à voir ...


array([  0.,   3.,  99.,  nan])

##### variables numériques :

In [99]:
lieux.dtypes
# le reste des variables sont des floats réellement ( donc on voit si y des des nan on remplace par mean)

Num_Acc      int64
catr       float64
voie        object
v1         float64
v2          object
circ       float64
nbv        float64
pr         float64
pr1        float64
vosp       float64
prof       float64
plan       float64
lartpc     float64
larrout    float64
surf       float64
infra      float64
situ       float64
env1       float64
dtype: object

In [100]:
lieux.isnull().sum()

Num_Acc         0
catr            0
voie        55754
v1         433094
v2         734917
circ            0
nbv          1148
pr         397330
pr1        398329
vosp       718719
prof            0
plan            0
lartpc       9005
larrout      7920
surf            0
infra        1110
situ            0
env1         1127
dtype: int64

###### voie :

In [101]:
lieux.voie.describe()
# 55754 nan
# contient beaucoup de 0 alors que 0 n'est pas un numéro de route 
# on transforme 0 à nan puis on voit le mode puis (nan==> le mode)

count     709822.0
unique      2469.0
top            0.0
freq      399947.0
Name: voie, dtype: float64

In [102]:
lieux.loc[:,['voie']]=lieux.loc[:,['voie']].replace(0,np.nan)


In [103]:
lieux.voie.describe()

count     309875.0
unique      2468.0
top            1.0
freq       19374.0
Name: voie, dtype: float64

In [104]:
lieux.loc[:,['voie']]=lieux.loc[:,['voie']].replace(np.nan,1)

###### Indice numériques du numéro de la route : 

In [105]:
lieux.v1.unique()

array([  0.,   2.,   1.,   9.,   6.,   4.,   5.,   7.,   3.,   8.,  nan])

In [106]:
lieux.v1.astype(str).describe() 
# à voir comment régler cette variable

count     765576
unique        11
top          nan
freq      433094
Name: v1, dtype: object

###### Lettre V2

In [107]:
lieux.v2.unique()
# y a une probléme de encoding : /x04
# y des nan 
# y des 0  
# il faut comprendre la variable pour 

array(['B', nan, 'A', 'E', 'N', 'D', 'C', 'F', 'R', 'G', 'J', 'b', 'a',
       'S', 'W', 'H', 'P', '\x04', 'K', 'T', 'L', 'M', 'Z', 'O', 'I', 's',
       'n', 'Q', 'e', 'V', 'd', 'c', 'z', 'U', 'v', 'm', 'Y', 'w', 'y',
       't', '0', '4', '1', '8', '2', '5', '6', '9', '3', '7', 'f', 'p',
       'g', 'l', 'r', 'X'], dtype=object)

###### nombre total de voie de circulation  :

In [108]:
print(sorted(set(list(lieux.nbv.unique()))))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, nan, 33.0, 36.0, 39.0, 40.0, 41.0, 42.0, 44.0, 45.0, 50.0, 52.0, 53.0, 54.0, 60.0, 62.0, 65.0, 70.0, 76.0, 84.0, 86.0, 90.0, 91.0, 99.0]


In [109]:
# on remplace nan par la moyenne ( méthode stationnaire d'imputation dans le cas des MCAR)
lieux.loc[:,['nbv']]=lieux.loc[:,['nbv']].replace(np.nan,int(lieux.nbv.mean())) #  on prends la partie entiére

###### Numéro PR :
    

In [110]:
lieux.loc[:,['pr']]=lieux.loc[:,['pr']].replace(np.nan,int(lieux.pr.mean())) 

###### Distance en mètres au PR :

In [111]:
lieux.loc[:,['pr1']]=lieux.loc[:,['pr1']].replace(np.nan,lieux.pr1.mean()) 

###### Largeur du terre plein central TPC ( s il exist )

In [112]:

lieux.loc[:,['lartpc']]=lieux.loc[:,['lartpc']].replace(np.nan,lieux.lartpc.mean()) 

###### larrout

In [113]:
lieux.loc[:,['larrout']]=lieux.loc[:,['larrout']].replace(np.nan,lieux.larrout.mean())

In [232]:
# change types: 
lieux.dtypes

Num_Acc      int64
catr       float64
voie        object
v1         float64
v2          object
circ       float64
nbv        float64
pr         float64
pr1        float64
vosp       float64
prof       float64
plan       float64
lartpc     float64
larrout    float64
surf       float64
infra      float64
situ       float64
env1       float64
dtype: object

In [ ]:
lieux.catr=lieux.catr.astpe(str)
lieux.circ=lieux.circ.astype(str)
lieux.vosp=lieux.vosp.astype(str)
lieux.prof=lieux.prof.astype(str)
lieux.plan=lieux.plan.astype(str)
lieux.surf=lieux.surf.astype(str)
lieux.infra=lieux.infra.astype(str)
lieux.situ=lieux.situ.astype(str)

In [ ]:
lieux.env1.unique()

In [114]:
    # save table to csv file
lieux.to_csv('lieux_2005-2016.csv',index=False)

---
#### la table usagers 
---

In [115]:
#shape :
usagers.shape

(1710043, 12)

In [116]:
print(usagers.dtypes)

Num_Acc      int64
place      float64
catu         int64
grav         int64
sexe         int64
trajet     float64
secu       float64
locp       float64
actp       float64
etatp      float64
an_nais    float64
num_veh     object
dtype: object


###### place : 

In [117]:
usagers.place.unique()

array([  1.,   2.,   4.,   5.,   3.,   0.,   8.,   9.,   7.,   6.,  nan])

In [118]:
print("taille 0 :" , usagers[usagers.place==0].shape)
print("taille nan :" , usagers.place.isnull().sum())


taille 0 : (60766, 12)
taille nan : 86734


In [119]:
#on les remplace par ? 
# ici les valeurs manquantes vont dependre de la nature du vehicule 
# donc pour chaque type de vehicule on va remplacer les valeurs manquates par le mode.
# donc on verra cette variable apès avoir fait la jointure entre les tables 
#sinon on peut simplifier et remplacer par le mode:
usagers.place.astype(str).describe()

count     1710043
unique         11
top           1.0
freq      1275547
Name: place, dtype: object

In [120]:
usagers.loc[:,['place']]=usagers.loc[:,['place']].replace(0,1)
usagers.loc[:,['place']]=usagers.loc[:,['place']].replace(np.nan,1)

###### Catégorie d'usager :

In [121]:
usagers.catu.unique()
#c'est bon

array([1, 2, 3, 4], dtype=int64)

###### Gravité de l'accident​ :

In [122]:
usagers.grav.unique()
#c'est bon

array([4, 3, 1, 2], dtype=int64)

###### Sexe de l'usager :


In [123]:
usagers.sexe.unique()
#c'est bon

array([1, 2], dtype=int64)

###### Annee de naissance de l'usager'

In [124]:
usagers.an_nais.dtype
# pas de soucis ici sauf si il y une erreur dans la saisie  d'une data :
print('min des dates :',usagers.an_nais.min())
print('max des dates :',usagers.an_nais.max())

min des dates : 1896.0
max des dates : 2016.0


Si les accidents ont lieu entre 2005-2016, alors un passer né en 1896 il a eu à la date d'accident en minimum un âge de 109 
et au max 120 ans. Ces deux date restent acceptable ~~

###### Motif du déplacement au moment de l’accident :

In [125]:
usagers.trajet.unique()

array([  1.,   3.,   0.,   5.,   9.,   4.,   2.,  nan])

In [126]:
print("taille 0 :" , usagers[usagers.trajet==0].shape)
print("taille nan :" , usagers.trajet.isnull().sum())

taille 0 : (506839, 12)
taille nan : 272


In [127]:
# on remplace par 9 :
usagers.loc[:,['trajet']]=usagers.loc[:,['trajet']].replace(0,9)
usagers.loc[:,['trajet']]=usagers.loc[:,['trajet']].replace(np.nan,9)

###### secu :

In [128]:
usagers.secu.unique()

array([ 11.,  31.,  21.,   0.,  12.,  13.,  90.,  10.,  22.,  91.,  33.,
        41.,  23.,  20.,  92.,  32.,  43.,  93.,  42.,   1.,  nan,  30.,
        40.,   3.,   2.])

In [129]:
secu_values=list(usagers.secu.astype(str).values)


In [130]:
# on traite d'abord le len puis les nan :
secu_values_set=[secu[:-2] for secu in secu_values if secu!='nan']
# d'après le document chaque valeur est censée d'avoir deux chiffres:

In [131]:
print(set(secu_values_set))
# il y le 0 et parfois on a qu'un seul chiffre ( ex 2, 1, 3)

{'1', '40', '91', '20', '12', '2', '3', '23', '42', '30', '21', '10', '33', '22', '31', '90', '11', '43', '32', '93', '13', '0', '41', '92'}


In [132]:
usagers.secu.astype(str).describe()
#top 11 : ceinture exist et elle est utilisée

count     1710043
unique         25
top          11.0
freq       952293
Name: secu, dtype: object

In [133]:
# remplacer nan et 0 par 11 :
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(0,11)
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(np.nan,11)

In [134]:
# pour le reste : soit on simplifie et on remplace par 11
# soit on compléte le chiffre par un deuxième qui soit le mode approprié au chiffre existant :
# deuxième méthode est plus logique :
from statistics import mode
secu_1=[secu for secu in secu_values if '1' in secu]
mode(secu_1)
# donc on remplace par 11


'11.0'

In [135]:
secu_2=[secu for secu in secu_values if '2' in secu]
mode(secu_2)
# on remplace par '21'

'21.0'

In [136]:
secu_3=[secu for secu in secu_values if '3' in secu]
mode(secu_3)
# on remplace par '13'

'13.0'

In [137]:
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(1,11)
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(2,21)
usagers.loc[:,['secu']]=usagers.loc[:,['secu']].replace(3,13)


###### Localisation du piéton :

In [138]:
usagers.locp.unique()

array([  0.,   2.,   4.,   1.,   5.,   6.,   3.,   8.,   7.,  nan])

In [139]:
print("taille 0 :" , usagers[usagers.locp==0].shape)
print("taille nan :" , usagers.locp.isnull().sum())
print(usagers.shape)

taille 0 : (1579313, 12)
taille nan : 1071
(1710043, 12)


In [140]:
# ici on va laisser 0 comme une autre valeur, il se peut qu'elle aie pour signification : 'il n'y a pas de piéton'
# on remplace nan par 0 également 
usagers.loc[:,['locp']]=usagers.loc[:,['locp']].replace(np.nan,0)

###### Action du piéton :

In [141]:
usagers.actp.unique()

array([  0.,   3.,   1.,   5.,   2.,   9.,   6.,   4.,  nan])

In [142]:
# nan ==> 0
usagers.loc[:,['locp']]=usagers.loc[:,['locp']].replace(np.nan,0)

###### etatp

In [143]:
usagers.etatp.unique()

array([  0.,   2.,   1.,   3.,  nan])

In [144]:
# on laisse 0 comme une autre valeur qui va indiquer qu'il n'y avait pas de piéton
# nan ==> 0
usagers.loc[:,['etatp']]=usagers.loc[:,['etatp']].replace(np.nan,0)

In [229]:
#change types from (int, float)==> object
usagers.place=usagers.place.astype(str)
usagers.catu=usagers.catu.astype(str)

usagers.grav=usagers.grav.astype(str)
usagers.sexe=usagers.sexe.astype(str)
usagers.trajet=usagers.trajet.astype(str)
usagers.secu=usagers.secu.astype(str)
usagers.locp=usagers.locp.astype(str)
usagers.actp=usagers.actp.astype(str)
usagers.etatp=usagers.etatp.astype(str)

Save to csv file :

In [145]:
usagers.to_csv('usager_2005-2016.csv', index=False)

---
la table des vehicules
---

In [146]:
vehicules.shape

(1307042, 9)

In [147]:
vehicules.dtypes


Num_Acc      int64
senc       float64
catv         int64
occutc       int64
obs        float64
obsm       float64
choc       float64
manv       float64
num_veh     object
dtype: object

###### Sens de circulation

In [148]:
vehicules.senc.unique()

array([  0.,   1.,   2.,  nan])

In [149]:
print("taille 0 :" , vehicules[vehicules.senc==0].shape)
print("taille nan :" , vehicules.senc.isnull().sum())

taille 0 : (1199403, 9)
taille nan : 72


In [150]:
vehicules.senc.astype(str).describe()

count     1307042
unique          4
top           0.0
freq      1199403
Name: senc, dtype: object

In [151]:
# on remplace par une valeur au choix :  sens croissant 
vehicules.loc[:,['senc']]=vehicules.loc[:,['senc']].replace(np.nan,1)
vehicules.loc[:,['senc']]=vehicules.loc[:,['senc']].replace(0,1)

###### Catégorie du véhicule :

In [152]:
print(len(vehicules.catv.unique()))
vehicules.catv.unique()
#Ok

33


array([ 7,  2,  9,  5, 18, 17,  1, 10,  4, 15, 19, 14, 13, 12, 21, 20, 99,
        3, 16,  8,  6, 11, 38, 32, 37, 30, 36, 31, 33, 39, 34, 35, 40], dtype=int64)

###### Obstacle mobile heurté :

In [153]:
vehicules.obsm.unique()

array([  2.,   0.,   1.,   9.,   4.,   5.,   6.,  nan])

In [154]:
# nan,0===>9
vehicules.loc[:,['obsm']]=vehicules.loc[:,['obsm']].replace(np.nan,9)
vehicules.loc[:,['obsm']]=vehicules.loc[:,['obsm']].replace(0,9)

###### Point de choc initial :

In [155]:
vehicules.choc.unique()

array([  1.,   8.,   7.,   0.,   3.,   2.,   6.,   4.,   5.,   9.,  nan])

In [156]:
# 0, nan ==> mode :
vehicules.choc.astype(str).describe()

count     1307042
unique         11
top           1.0
freq       485214
Name: choc, dtype: object

In [157]:
vehicules.loc[:,['choc']]=vehicules.loc[:,['choc']].replace(np.nan,1)
vehicules.loc[:,['choc']]=vehicules.loc[:,['choc']].replace(0,1)

###### Nombre d’occupants dans le transport en commun

In [158]:
vehicules.occutc.unique()
# 0 pour dire que ce n'est pas transport commun.

array([  0,   1,   2,  26,  23,  36,  31,   6,  51,  27,  52,  25,  10,
         8,  14,  20,  11,  40,  33,  12,   3,  13,  45,  39,   5,  15,
        32, 100,  30,  29,  35,  41,   4,  21,  16,  18,  37,  60,  62,
        46,  50,  28,  44,   7,  57,  42,   9,  17,  58,  24,  43,  48,
        22, 470,  69,  61, 150,  47, 120,  66,  49,  38,  54,  59, 200,
        34,  53,  55,  96,  65, 210,  63,  93,  80,  85, 213,  64, 193,
       111,  70, 700, 300,  73,  74,  19,  72, 110,  97,  56,  75, 220,
        81,  95,  84,  83,  67,  76,  89,  90, 480, 140,  99, 240,  71,
       132, 125, 153, 184, 170, 130, 490, 250], dtype=int64)

###### manv

In [159]:
# manv de type int
vehicules.manv.isnull().sum()

227

In [160]:
vehicules.manv.astype(str).describe()

count     1307042
unique         26
top           1.0
freq       575354
Name: manv, dtype: object

In [161]:
vehicules.loc[:,['manv']]=vehicules.loc[:,['manv']].replace(np.nan,1)

In [230]:
vehicules.dtypes

Num_Acc      int64
senc       float64
catv         int64
occutc       int64
obs        float64
obsm       float64
choc       float64
manv       float64
num_veh     object
dtype: object

In [ ]:
vehicules.senc=vehicules.senc.astype(str)
vehicules.catv=vehicules.catv.astype(str)
vehicules.obs=vehicules.obs.astype(str)
vehicules.obsm=vehicules.obsm.astype(str)
vehicules.choc=vehicules.choc.astype(str)
vehicules.manv=vehicules.manv.astype(str)


save to csv file 

In [162]:
vehicules.to_csv('vehicules_2005-2016.csv', index=False)

---
Fin nettoyage 
---

### La jointure des tables :

In [163]:
#vehicules.head(5)

In [164]:
#usagers.head(6)

In [165]:
#print(lieux.shape)
#print(caracteristiques.shape) 


#on commence par ces deux table. PK/FK est Num_Acc

In [166]:
# je choisi que 90 000 ligne pour le test ( et d'ailler ca passe pas la taille entière sur mon pc)
#carac_lieux_marged=pd.merge(caracteristiques.iloc[:90000,:],lieux.iloc[:90000,:],on='Num_Acc')

In [167]:
#carac_lieux_marged.head()

on merge avec vehicules : 

vehicules.head()

carac_lieux_veh_merged=pd.merge(carac_lieux_marged, vehicules,on='Num_Acc' )

carac_lieux_veh_merged.head()

save to csv file 

carac_lieux_veh_merged.to_csv("carac_lieux_veh_merged.csv", index=False)

usagers.shape

usagers.head(20)

on merge finalement avec usagers :

In [168]:
#  later 
#final_data=pd.merge(carac_lieux_veh_merged,usagers,on=['num_veh','num_veh'])

# Model de prédiction de la gravité des accidents avec les données 2015 :

In [26]:
cara2015=pd.read_csv('dataset/caracteristiques/caracteristiques_2015.csv',encoding='latin-1')
vehi2015=pd.read_csv('dataset/vehicules/vehicules_2015.csv',encoding='latin-1')
lieux2015=pd.read_csv('dataset/lieux/lieux_2015.csv',encoding='latin-1')
usager2015=pd.read_csv('dataset/usagers/usagers_2015.csv',encoding='latin-1')



In [27]:
merge1=pd.merge(cara2015,lieux2015,on='Num_Acc')

In [28]:
merge2=pd.merge(merge1,vehi2015,on='Num_Acc')

In [ ]:
data=pd.merge(merge2,usager2015,on=['Num_Acc','num_veh'])

In [39]:
len(data.columns)

51

In [40]:
data.columns

Index(['Num_Acc', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm',
       'col', 'com', 'adr', 'gps', 'lat', 'long', 'dep', 'catr', 'voie', 'v1',
       'v2', 'circ', 'nbv', 'pr', 'pr1', 'vosp', 'prof', 'plan', 'lartpc',
       'larrout', 'surf', 'infra', 'situ', 'env1', 'senc', 'catv', 'occutc',
       'obs', 'obsm', 'choc', 'manv', 'num_veh', 'place', 'catu', 'grav',
       'sexe', 'trajet', 'secu', 'locp', 'actp', 'etatp', 'an_nais'],
      dtype='object')

In [72]:
col=['jour','int','col','lum','surf','infra','sexe','trajet','place','agg','catv','grav']
data2=data.loc[:,col]

In [73]:
data2.columns

Index(['jour', 'int', 'col', 'lum', 'surf', 'infra', 'sexe', 'trajet', 'place',
       'agg', 'catv', 'grav'],
      dtype='object')

In [74]:
final_data=data2.dropna()

In [75]:
final_data.shape

(118111, 12)

In [76]:
final_data=final_data.replace(0,np.nan)

In [77]:
final_data=final_data.dropna()

In [78]:
final_data.shape

(9429, 12)

In [79]:
Y=final_data.grav.values

In [80]:
X=final_data.loc[:,final_data.columns!='grav']


In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train1, X_test1,Y_train1,Y_test1 = train_test_split(X, Y, test_size=0.33, random_state=99)
#Without weather
svc = SVC()
svc.fit(X_train1, Y_train1)
Y_pred = svc.predict(X_test1)
acc_svc1 = round(svc.score(X_test1, Y_test1) * 100, 2)
acc_svc1

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train1, Y_train1)
Y_pred = knn.predict(X_test1)
acc_knn1 = round(knn.score(X_test1, Y_test1) * 100, 2)
acc_knn1


# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train1, Y_train1)
Y_pred = logreg.predict(X_test1)
acc_log1 = round(logreg.score(X_train1, Y_train1) * 100, 2)
acc_log1


# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train1, Y_train1)
Y_pred = gaussian.predict(X_test1)
acc_gaussian1 = round(gaussian.score(X_test1, Y_test1) * 100, 2)
acc_gaussian1

# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train1, Y_train1)
Y_pred = perceptron.predict(X_test1)
acc_perceptron1 = round(perceptron.score(X_test1, Y_test1) * 100, 2)
acc_perceptron1

# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train1, Y_train1)
Y_pred = linear_svc.predict(X_test1)
acc_linear_svc1 = round(linear_svc.score(X_test1, Y_test1) * 100, 2)
acc_linear_svc1

# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train1, Y_train1)
Y_pred = sgd.predict(X_test1)
acc_sgd1 = round(sgd.score(X_test1, Y_test1) * 100, 2)
acc_sgd1

# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train1, Y_train1)
Y_pred = decision_tree.predict(X_test1)
acc_decision_tree1 = round(decision_tree.score(X_test1, Y_test1) * 100, 2)
acc_decision_tree1

# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train1, Y_train1)
Y_pred = random_forest.predict(X_test1)
random_forest.score(X_train1, Y_train1)
acc_random_forest1 = round(random_forest.score(X_test1, Y_test1) * 100, 2)
acc_random_forest1

55.619999999999997

In [82]:
print("Machine Learning algorithm scores without weather related conditions")
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc1, acc_knn1, acc_log1, 
              acc_random_forest1, acc_gaussian1, acc_perceptron1, 
              acc_sgd1, acc_linear_svc1, acc_decision_tree1]})
models.sort_values(by='Score', ascending=False)

Machine Learning algorithm scores without weather related conditions


,Model,Score
0,Support Vector Machines,56.65
3,Random Forest,55.62
4,Naive Bayes,53.79
2,Logistic Regression,53.73
1,KNN,53.15
8,Decision Tree,49.78
7,Linear SVC,48.49
6,Stochastic Gradient Decent,46.43
5,Perceptron,15.62


In [83]:
# Confusion matrix with random forest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
x,y = final_data.loc[:,final_data.columns != 'grav'], final_data.loc[:,'grav']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 1)
rf = RandomForestClassifier(random_state = 4)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)
print('Classification report: \n',classification_report(y_test,y_pred))
y_test.value_counts()

Confusion matrix: 
 [[981   2  75 210]
 [ 17   4  29  12]
 [158   5 222 173]
 [387   4 200 350]]
Classification report: 
              precision    recall  f1-score   support

          1       0.64      0.77      0.70      1268
          2       0.27      0.06      0.10        62
          3       0.42      0.40      0.41       558
          4       0.47      0.37      0.42       941

avg / total       0.53      0.55      0.53      2829



1    1268
4     941
3     558
2      62
Name: grav, dtype: int64

In [84]:
rf.feature_importances_


array([ 0.27963494,  0.08787513,  0.11561664,  0.05721408,  0.03624301,
        0.06961284,  0.02836292,  0.07669173,  0.03614986,  0.02593936,
        0.18665948])

In [85]:
print(col)

['jour', 'int', 'col', 'lum', 'surf', 'infra', 'sexe', 'trajet', 'place', 'agg', 'catv', 'grav']


# Jointure année par année

In [65]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import pandas as pd
caracteristiques=pd.read_csv('caracteristiques_2005-2016.csv',encoding='latin-1')


In [2]:
vehicules=pd.read_csv('vehicules_2005-2016.csv',encoding='latin-1')

In [3]:
lieux=pd.read_csv('lieux_2005-2016.csv',encoding='latin-1')

C:\Users\pc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
usagers=pd.read_csv('usager_2005-2016.csv',encoding='latin-1')

In [ ]:
dfs=[]
col=[2005,2006,2007,2008,2010,2011,2012,2013,2014,2015,2016]
for year in col :
    c=caracteristiques.loc[caracteristiques['Num_Acc']//100000000==year]
    v=vehicules.loc[vehicules['Num_Acc']//100000000==year]
    u=usagers.loc[usagers['Num_Acc']//100000000==year]
    l=lieux.loc[lieux['Num_Acc']//100000000==year]
    merge1=pd.merge(c,l,on='Num_Acc')
    merge2=pd.merge(merge1,v,on='Num_Acc')
    data=pd.merge(merge2,u,on=['Num_Acc','num_veh'])
    dfs.append(data)
    

    
    
    
    

In [20]:
for index, item in enumerate(dfs, start = 1):
    item.to_csv('data'+str(index)+'.csv')

# Modèle de prédiction de la gravité des accidents :

### Suivant les années :

##### Load data : 

In [16]:
final_data=pd.DataFrame()
for i in range(1,11):
    df=pd.read_csv('data'+str(i)+'.csv',encoding='latin-1')
    df= df.loc[(np.isfinite(df.long) & np.isfinite(df.lat)),:]
    final_data=pd.concat([final_data,df])


    

C:\Users\pc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##### Describe data :

In [19]:
print("Shape of our data is :",final_data.shape)
print("We have {} columns in our data".format(len(final_data.columns)))

Shape of our data is : (650494, 52)
We have 52 columns in our data


##### Save data as csv file:

In [22]:
final_data.to_csv('data_cleaned.csv')

In [23]:
%reset  # clean memory

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


##### Load csv file : datacleaned

In [26]:
import pandas as pd
df=pd.read_csv('data_cleaned.csv',encoding='latin-1')


In [28]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Num_Acc,an,mois,jour,hrmn,lum,agg,int,...,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais
0,0,0,200500000001,5,1,12,1900,3,2,1,...,1.0,1,4,1,1.0,11.0,0.0,0.0,0.0,1976.0
1,1,1,200500000001,5,1,12,1900,3,2,1,...,1.0,1,3,2,3.0,11.0,0.0,0.0,0.0,1968.0
2,2,2,200500000001,5,1,12,1900,3,2,1,...,2.0,2,1,1,9.0,11.0,0.0,0.0,0.0,1964.0
3,3,3,200500000001,5,1,12,1900,3,2,1,...,4.0,2,1,1,9.0,31.0,0.0,0.0,0.0,2004.0
4,4,4,200500000001,5,1,12,1900,3,2,1,...,5.0,2,1,1,9.0,11.0,0.0,0.0,0.0,1998.0


##### Column used for classification 

In [38]:
col_used=[  'jour', 'lum', 'agg', 'int', 'atm', 'col',
           'catr', 'voie', 'circ', 'nbv', 'pr', 'pr1', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 
           'situ', 'senc', 'catv', 'occutc', 'obsm', 'choc', 'manv', 'place', 'catu', 'grav', 'sexe',
           'trajet','secu', 'locp', 'etatp']

##### Data for trainin using columns defined above :

In [39]:
train=df.loc[:,col_used]

In [49]:
# write to csv file
train.to_csv("training.csv")

In [57]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [58]:
import pandas as pd
train=pd.read_csv('training.csv',encoding='latin-1')

In [41]:

#check for missing data
train.isnull().sum().sum()

0

In [59]:
Y=train.grav.values
X=train.loc[:,train.columns!='grav']


In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Confusion matrix with random forest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
#x,y = final_data.loc[:,final_data.columns != 'grav'], final_data.loc[:,'grav']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3,random_state = 1)
rf = RandomForestClassifier(random_state = 4)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)
print('Classification report: \n',classification_report(y_test,y_pred))
y_test.value_counts()

In [55]:
rf.feature_importances_

array([ 0.11439214,  0.07569155,  0.02172624,  0.01784444,  0.01720044,
        0.01883931,  0.04012812,  0.02112252,  0.06652797,  0.01286454,
        0.02767372,  0.05466703,  0.04626404,  0.01836548,  0.01924391,
        0.0185654 ,  0.06518977,  0.01615345,  0.01307017,  0.00041953,
        0.04829138,  0.00164684,  0.03327356,  0.03790833,  0.04391337,
        0.01777471,  0.01609464,  0.01296916,  0.02899997,  0.05520325,
        0.00859978,  0.00937521])